First, importing all packages which will be necessary through the process.

In [1]:
import pandas as pd
import numpy as np
!pip install geocoder
import geocoder
import geopy
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import requests 
from geopy.geocoders import Nominatim

Solving environment: done

# All requested packages already installed.



In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

Using the read_html pandas feature, I have read the HTML tables on the Wikipedia page and then took the first one, which consists of Postal Code, Boroughs and Neighborhoods.

In [3]:
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header=0)
tables = tables[0]
tables.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Doing the proper adjusts to the table, by dropping Rows with Borough not assigned, replacing "/" by "," and changing any Neighborhood which has "Not Assigned" to the name of its Borough.

In [4]:
# Get columns with borough = Not assigned
indexNames = tables[ tables['Borough'] == 'Not assigned' ].index

# Drop columns
tables.drop(indexNames , inplace=True)

#Replace "/" with ","
tables = tables.stack().str.replace(' / ',', ').unstack()
tables.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [ ]:
neighborhoods = tables['Neighborhood']

for x in neighborhoods:
    if x == 'Not assigned':
        tables['Neighborhood'] = tables['Borough']
        
tables.head()
tables.shape

(103, 3)

Grabbing the Latitude and Longitude data from the csv and then merging it with our previous DataFrame.

In [ ]:
latlon=pd.read_csv(r'https://cocl.us/Geospatial_data')
latlon = latlon.rename(columns={'Postal Code': 'Postal code'})
latlon.head()


,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [ ]:
torontolatlon = pd.merge(tables,latlon)
torontolatlon.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [ ]:
address = 'Toronto'
geolocator = Nominatim(user_agent="exploring_toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Now with the data well formatted, next step is to grab data from Foursquare and add it to our Dataframe, in order to be possible to analyze it with K-Means.

In [ ]:
CLIENT_ID = 'EX3NXMNX5TLNEGG5KTIXLABHKWAIQZP4WBMG4CPGTSCD0DQ4' # your Foursquare ID
CLIENT_SECRET = 'BFVHVLQ352ZNOHCBJUMSCZ2SNGZOV34TQX5TWQ2D1SHVX5NT' # your Foursquare Secret
VERSION = '20200101' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
torontovenues = getNearbyVenues(names=torontolatlon['Neighborhood'],
                                   latitudes=torontolatlon['Latitude'],
                                   longitudes=torontolatlon['Longitude']
                                  )



Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government


In [ ]:
print(torontovenues.shape)
torontovenues

In [ ]:
torontovenues.groupby('Neighborhood').count()

In [ ]:
torontop = pd.get_dummies(torontovenues[['Venue Category']], prefix="", prefix_sep="")
torontop['ZNeighborhood'] = torontovenues['Neighborhood'] 
rightcolumns = [torontop.columns[-1]] + list(torontop.columns[:-1])
torontop = torontop[rightcolumns]
torontop = torontop.rename(columns={'ZNeighborhood':'Correct Neighborhood'})

torontop.head()

In [ ]:
torontogroup = torontop.groupby('Correct Neighborhood').mean().reset_index()
torontogroup
torontogroup.shape

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = torontogroup['Correct Neighborhood']

for ind in np.arange(torontogroup.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(torontogroup.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Since I decided to take the whole city, not only those Boroughs with "Toronto" on its name, 5 clusters wouldn't define well the city - since it's way bigget than Manhattan. Thus, 7 clusters looked ideal.

In [ ]:
# set number of clusters
kclusters = 7

torontogroupcluster = torontogroup.drop('Correct Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(torontogroupcluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

torontomerge = torontolatlon

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
torontomerge = torontomerge.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')



In [ ]:
torontomerge= torontomerge.dropna()
torontomerge['Cluster Labels'] = torontomerge['Cluster Labels'].apply(np.int64)
torontomerge

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(torontomerge['Latitude'], torontomerge['Longitude'], torontomerge['Neighborhood'], torontomerge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters